In [9]:
import heapq
import osmnx as ox
import math
import pickle

In [10]:
f = open('blr.bin', 'rb')
print('Graph found, loading from disk')
G = pickle.load(f)
f.close()

Graph found, loading from disk


In [25]:
class a_star:
    def __init__(self, graph):
        self.graph = graph

    def get_route(self, pair1, pair2):
        self.start_node = ox.distance.nearest_nodes(G, pair1[1], pair1[0])
        self.goal_node = ox.distance.nearest_nodes(G, pair2[1], pair2[0])

    def calculate_route(self):
        # Create a priority queue and add the start node with a cost of 0
        pq = [(0, self.start_node)]
        # Create a dictionary to keep track of the cost to reach each node
        cost_so_far = {self.start_node: 0}
        # Create a dictionary to keep track of the parent of each node
        parent = {self.start_node: None}
        
        while pq:
            # Pop the node with the lowest cost from the priority queue
            _, current_node = heapq.heappop(pq)
            
            # If we have reached the goal, reconstruct the path and return it
            if current_node == self.goal_node:
                path = []
                while current_node:
                    path.append(current_node)
                    current_node = parent[current_node]
                return path[::-1]
            
            # Otherwise, expand the current node's neighbors and add them to the priority queue
            for neighbor_node in self.neighbors(current_node):
                # Calculate the cost to reach the neighbor node
                new_cost = cost_so_far[current_node] + 1
                # If we haven't visited this neighbor node yet or the new cost is lower than the old cost, update the cost and parent
                if neighbor_node not in cost_so_far or new_cost < cost_so_far[neighbor_node]:
                    cost_so_far[neighbor_node] = new_cost
                    priority = new_cost + self.heuristic_cost_estimate(neighbor_node, self.goal_node)
                    heapq.heappush(pq, (priority, neighbor_node))
                    parent[neighbor_node] = current_node
        
        # If we get here, there is no path from the start node to the goal node
        return None
    def haversine(self, pair1, pair2):
        """
        Calculate the great circle distance between two points
        on the earth (specified in decimal degrees)
        """
        # convert decimal degrees to radians
        lon1, lat1, lon2, lat2 = map(math.radians, [pair1[1], pair1[0], pair2[1], pair2[0]])

        # haversine formula
        dlon = lon2 - lon1
        dlat = lat2 - lat1
        a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
        c = 2 * math.asin(math.sqrt(a))
        r = 6371 # Radius of earth in kilometers. Use 3956 for miles
        return c * r
    
    def heuristic_cost_estimate(self, n1, n2) -> float:
        if isinstance(n1, int):
            n1 = self.graph.nodes[n1]['x'], self.graph.nodes[n1]['y']
        if isinstance(n2, int):
            n2 = self.graph.nodes[n2]['x'], self.graph.nodes[n2]['y']
        x1, y1 = n1
        x2, y2 = n2
        return self.haversine((y1, x1), (y2, x2))

    def neighbors(self, node):
        return list(self.graph.neighbors(node))

In [13]:
source_coord = (12.909229431138986, 77.63947874679494)
dest_coord = (12.914080814688852, 77.64155496935096)

In [14]:
start_node = ox.distance.nearest_nodes(G, source_coord[1], source_coord[0])
goal_node = ox.distance.nearest_nodes(G, dest_coord[1], dest_coord[0])

In [27]:
routingObj = a_star(G)
path = routingObj.get_route(start_node, goal_node)

In [28]:
for i in path:
    lon = G._node[i]['x'] #lon
    lat = G._node[i]['y'] #lat
    print([lat, lon])

[12.9092598, 77.6395172]
[12.9092659, 77.6391683]
[12.9092789, 77.6384189]
[12.9092848, 77.6380325]
[12.9097988, 77.638043]
[12.9097451, 77.6416337]
[12.9104182, 77.6416628]
[12.9114151, 77.6417058]
[12.9119965, 77.6417201]
[12.9122085, 77.6417126]
[12.9121971, 77.64204]
[12.912791, 77.6420643]
[12.9127933, 77.6414959]
[12.9140982, 77.6415886]
